# WWW: Who Will Win?

## 12 April, 2018

"It's tough to make predictions, especially [about](https://en.wikiquote.org/wiki/Yogi_Berra)  [the](https://en.wikiquote.org/wiki/Niels_Bohr) future." That's true for the **NBA basketball playoffs**, where the Rockets are the favorites, with a 44% chance of winning the title according to [538](https://fivethirtyeight.com/features/the-nba-playoffs-sleepers-favorites-and-best-first-round-matchups/), and 35% according to the Las Vegas oddsmakers, a relatively small difference of opinion. But there are some big discrepancies: the Warriors are tied with the Rockets at 35% according to Vegas, but stand at only 4% according to 538. That 9-fold difference underscores that rational people can use different models with different assumptions and come to different conclusions.  Here are some models you might choose:

1. **Holistic**: "I just feel that the Rockets have about a 1/3 chance of winning it all."
2. **Game by Game**: "I think the Rockets have an 75% chance of winning each game in the first round, then 70% in the next round, then 60%; from that I'll calculate their overall chance."
3. **Point by Point**: "The Rockets have a per-game average point differential of 8.2; I'll compare that to the other teams and caclulate their overall chance."

# Point by Point Model

The [Simple Rating System](https://www.sportingcharts.com/dictionary/nba/simple-rating-system-statistics.aspx) (SRS) records the average point differential of a team over the season, with a slight adjustment for strength of schedule.  We can look up the winning percentage and SRS of the top contending teams  [here](https://www.basketball-reference.com/leagues/NBA_2018.html): 

     TEAM      PCT  SRS
     Rockets  .793  8.2
     Raptors  .720  7.3
     Warriors .707  5.8
     Sixers   .634  4.3
     

The Point-by-Point model of a game is: a game is decided by a random sample from the distribution of point differentials, which is a normal (Gaussian) distribution centered around the recorded differences of the two teams. So, if the Raptors play the Sixers, then the
mean of this distribution is 7.3 - 4.3 = 3.0. We also need to know the standard deviation; [Betlabs](https://www.betlabssports.com/blog/a-look-at-nba-team-totals/) says it
is 10.5 points (and they confirm that scores roughly follow a normal  distribution). 
The function `simulate` does the calculation:

In [1]:
from statistics import mean
from random import gauss

def simulate(diff, 𝝈=10.5, n=10000):
    "Given SRS point differential of two teams, return favored team's win probability."
    return mean(gauss(diff, 𝝈) > 0 for game in range(n))

In [2]:
simulate(7.3 - 4.3) # Raptors win probability vs Sixers

The model says the Raptors have a 61% chance of beating the Sixers in a game. We can make a table of point differentials and game win percentages:

In [3]:
pct = '{:4.0%}'.format 

for diff in range(10):
    print(diff, 'point differential =', pct(simulate(diff)), 'win game')

0 point differential =  51% win game
1 point differential =  53% win game
2 point differential =  58% win game
3 point differential =  61% win game
4 point differential =  66% win game
5 point differential =  68% win game
6 point differential =  72% win game
7 point differential =  75% win game
8 point differential =  78% win game
9 point differential =  80% win game


This agrees very well with the "Differential vs. Win Percentage" [chart](http://a.espncdn.com/combiner/i?img=%2Fphoto%2F2018%2F0408%2F180408_differential.png&w=1140&cquality=40) on [this page](http://www.espn.com/nba/story/_/id/23071005/kevin-pelton-weekly-mailbag-including-nba-all-offensive-teams).

# Game by Game Model

The next model says that a playoff series is a sequence of independent and identically distributed game results (where the probability of a single-game win is specified either using point differential, or holistically). The idea here is to be consistent: if you believe that a team's win percentage is 60%, and you believe that games are independent, then you must believe that the team's chance of wining 4 in a row is 0.6<sup>4</sup> = 0.1296.

The function `win_series` calculates the probability of winning a series, given the probability of winning a game:

In [4]:
def win_series(p, W=0, L=0):
    """Probability of winning best-of-7 series, given a probability p of winning a game.
    The optional arguments say how many Wins and Losses the team has in the series so far."""
    return (1 if W == 4 else
            0 if L == 4 else
            p * win_series(p, W + 1, L) + (1 - p) * win_series(p, W, L + 1))

We can make another table:

In [5]:
for p in range(50, 95, 5):
    print(format('{}% win game = {:5.1%} win series'.format(pct, win_series(p/100))))

<built-in method format of str object at 0x1035d5260>% win game = 50.0% win series
<built-in method format of str object at 0x1035d5260>% win game = 60.8% win series
<built-in method format of str object at 0x1035d5260>% win game = 71.0% win series
<built-in method format of str object at 0x1035d5260>% win game = 80.0% win series
<built-in method format of str object at 0x1035d5260>% win game = 87.4% win series
<built-in method format of str object at 0x1035d5260>% win game = 92.9% win series
<built-in method format of str object at 0x1035d5260>% win game = 96.7% win series
<built-in method format of str object at 0x1035d5260>% win game = 98.8% win series
<built-in method format of str object at 0x1035d5260>% win game = 99.7% win series


For example, if a team has a 60% chance of winning each game, then it has an 71% chance of winning the series. This model ignores the fact that games aren't strictly independent, and ignores home court advantage. Why? Because these factors would change the final winning estimate by only a few percentage points, and I already have more uncertainty than that. 

What happens if some games in a series have already been played? The following function prints a table where each row tells a team's current win-loss record, each column is the game win percentage, and each entry in the table is the series win percentage.

In [6]:
def series_table(pcts=[p/100 for p in range(20, 85, 5)]):
    print('W-L | Game Win Percentage')
    print('    | ' + ' '.join(map(pct, pcts)))
    for W in range(4):
        print('----+' + '-' * 5 * len(pcts))
        for L in range(4):
            results = [win_series(p, W, L) for p in pcts]
            print('{}-{} | {}'.format(W, L, ' '.join(map(pct, results))))

series_table()

W-L | Game Win Percentage
    |  20%  25%  30%  35%  40%  45%  50%  55%  60%  65%  70%  75%  80%
----+-----------------------------------------------------------------
0-0 |   3%   7%  13%  20%  29%  39%  50%  61%  71%  80%  87%  93%  97%
0-1 |   2%   4%   7%  12%  18%  26%  34%  44%  54%  65%  74%  83%  90%
0-2 |   1%   2%   3%   5%   9%  13%  19%  26%  34%  43%  53%  63%  74%
0-3 |   0%   0%   1%   2%   3%   4%   6%   9%  13%  18%  24%  32%  41%
----+-----------------------------------------------------------------
1-0 |  10%  17%  26%  35%  46%  56%  66%  74%  82%  88%  93%  96%  98%
1-1 |   6%  10%  16%  24%  32%  41%  50%  59%  68%  76%  84%  90%  94%
1-2 |   3%   5%   8%  13%  18%  24%  31%  39%  48%  56%  65%  74%  82%
1-3 |   1%   2%   3%   4%   6%   9%  12%  17%  22%  27%  34%  42%  51%
----+-----------------------------------------------------------------
2-0 |  26%  37%  47%  57%  66%  74%  81%  87%  91%  95%  97%  98%  99%
2-1 |  18%  26%  35%  44%  52%  61%  69%  76%  82% 

For example, if our team with a 60% win percentage loses the first game of the series, then check the "0-1" row and the "60%" column to see that it still has a 54% chance of winning the series. You also have the option of updating your prior probability. Suppose after seeing our 60% team lose the first game you decide "perhaps I was wrong about them; perhaps they're actually a 55% team". Then you would look at the 55% column of the 0-1 row and find that their chances have slipped to 44%.

# Series by Series Model

Here's a function to tabulate a team's chance of winning each series on the way to a title. The first argument is the team's name, and the second, `rounds` is a list of playoff round entries, where each entry is a tuple of the (expected) opponent team name, the game win percentage against this team, and optionally the wins and losses in the series so far:

In [7]:
def playoffs(name, rounds):
    "Print probability for team winning each series."
    overall = 1.0 
    for (opponent, p, *WL) in rounds:
        this_round = win_series(p, *WL)
        overall *= this_round
        print('{} vs {:8} {}; through here: {}'
              .format(name, opponent, pct(this_round), pct(overall)))

Here my holistic game estimates are turned into overall predictions:

In [8]:
playoffs('Rockets',
     [('Wolves',   0.75),
      ('Jazz',     0.70),
      ('Warriors', 0.60),
      ('Raptors',  0.60)])

Rockets vs Wolves    93%; through here:  93%
Rockets vs Jazz      87%; through here:  81%
Rockets vs Warriors  71%; through here:  58%
Rockets vs Raptors   71%; through here:  41%


So I'm in good agreement with [538](https://fivethirtyeight.com/features/the-nba-playoffs-sleepers-favorites-and-best-first-round-matchups/): I have the Rockets at 58% winning the conference and 41% winning the title, while 538 had them at  57% and 44%.

What if I went by point differential?

In [9]:
playoffs('Rockets',
     [('Wolves',   simulate(8.2 - 2.3)),
      ('Jazz',     simulate(8.2 - 4.5)),
      ('Warriors', simulate(8.2 - 5.8)),
      ('Raptors',  simulate(8.2 - 7.3))])

Rockets vs Wolves    88%; through here:  88%
Rockets vs Jazz      79%; through here:  69%
Rockets vs Warriors  70%; through here:  49%
Rockets vs Raptors   58%; through here:  28%


This puts the Rockets at 48% and 28%, which is closer to the Vegas odds of 44% and 35%. How do I reconcile this discrepancy? I guess I would say that I don't have  much faith in the point differential model, because it places too  much emphasis on meaningless scores: for example, in the Warriors' final game, it was to their strategic advantage to lose, and they did&mdash;by 40 points, which dropped their average differential for the entire year by 0.5 points.

Here are my projections for the hometown Warriors:

In [10]:
playoffs('Warriors',
     [('Spurs',    0.70),
      ('Portland', 0.65),
      ('Rockets',  0.40),
      ('Raptors',  0.55)])

Warriors vs Spurs     87%; through here:  87%
Warriors vs Portland  80%; through here:  70%
Warriors vs Rockets   29%; through here:  20%
Warriors vs Raptors   61%; through here:  12%


So I'm splitting the difference between 538's low estimate and Vegas's high estimate of the Warriors chances.

---

# 2016 NBA Playoffs


An historical record of the 2016 playoffs.

## 18 April 2016

The Golden State Warriors have had a historic basketball season, winning more games than any other team ever has. But will they top that off by winning the championship? There are 15 other teams in contention, including one, the Spurs, that has had a historic season as the best second-best team ever. The web site fivethirtyeight, using a complicated scoring system, [gives](http://projects.fivethirtyeight.com/2016-nba-picks/) the Warriors a 44% chance of winning, with the Spurs at 28%. Basketball-reference [has](http://www.basketball-reference.com/friv/playoff_prob.cgi) the Warriors at 41% and Spurs at 32.5%, while a [betting site](http://www.oddsshark.com/nba/nba-futures) had the Warriors at 54% and Spurs at 18%. 
  
Here are the top four teams with their stats:

     TEAM         PCT    SRS
     Warriors    .890  10.38
     Spurs       .817  10.28
     Thunder     .671   7.09
     Cavaliers   .695   5.45

I came up with my own subjective game winning percentages, informed by SRS calculations, plus injury status, etc.

In [11]:
playoffs('Warriors',
     [('Rockets',  0.83),
      ('Clippers', 0.73),
      ('Spurs',    0.58),
      ('Cavs',     0.67)])

Warriors vs Rockets   98%; through here:  98%
Warriors vs Clippers  91%; through here:  89%
Warriors vs Spurs     67%; through here:  60%
Warriors vs Cavs      83%; through here:  50%


In [12]:
playoffs('Spurs',
     [('Memphis',  0.83),
      ('Thunder',  0.62),
      ('Warriors', 0.42),
      ('Cavs',     0.67)])

Spurs vs Memphis   98%; through here:  98%
Spurs vs Thunder   75%; through here:  73%
Spurs vs Warriors  33%; through here:  24%
Spurs vs Cavs      83%; through here:  20%


In [13]:
playoffs('Cavs',
     [('Pistons', 0.83),
      ('Hawks',   0.60),
      ('Raptors', 0.55),
      ('GSW/SAS', 0.33)])

Cavs vs Pistons   98%; through here:  98%
Cavs vs Hawks     71%; through here:  70%
Cavs vs Raptors   61%; through here:  42%
Cavs vs GSW/SAS   17%; through here:   7%


I have the Warriors at 50% (for the estimate of winning it all) and the Spurs at 20%, so I'm more of a Warriors fan than fivethirtyeight and basketball-reference. Interestingly, while fivethirtyeight does not think this year's Warriors are better than the 1995 Bulls, they [do think](http://fivethirtyeight.com/features/the-warriors-still-arent-the-best-team-ever/) the Spurs, Thunder, and Cavs are the best ever second-, third-, and fourth-best teams in a season.

# WWWWC: Will Warriors Win Without Curry?

## 27 April 2016

The Playoff picture has changed! 

We have some results for first-round series, and there have been key injuries to players including Steph Curry, Avery Bradley, Chris Paul, and Blake Griffin. We don't know for sure how long Curry will be out, but here are my updated odds for the Warriors, under the assumption that Curry misses the second round, and comes back in time for the Western Conference Finals at a mildly reduced capacity:

In [14]:
playoffs('Warriors',
     [('Rockets',  0.70, 3, 1),
      ('Blazers',  0.55),
      ('Spurs',    0.55),
      ('Cavs',     0.60)])

Warriors vs Rockets   97%; through here:  97%
Warriors vs Blazers   61%; through here:  59%
Warriors vs Spurs     61%; through here:  36%
Warriors vs Cavs      71%; through here:  26%


The Spurs and Cavs are rolling; let's update their odds:

In [15]:
playoffs('Spurs',
     [('Memphis',  0.83, 4, 0),
      ('Thunder',  0.62),
      ('Warriors', 0.45),
      ('Cavs',     0.67)])

Spurs vs Memphis  100%; through here: 100%
Spurs vs Thunder   75%; through here:  75%
Spurs vs Warriors  39%; through here:  29%
Spurs vs Cavs      83%; through here:  24%


In [16]:
playoffs('Cavs',
     [('Pistons', 0.83, 4, 0),
      ('Hawks',   0.60),
      ('Raptors', 0.55),
      ('GSW/SAS', 0.40)])

Cavs vs Pistons  100%; through here: 100%
Cavs vs Hawks     71%; through here:  71%
Cavs vs Raptors   61%; through here:  43%
Cavs vs GSW/SAS   29%; through here:  13%


So my updated odds are that the Warriors and Spurs are roughly equally likely to win (26% and 24%); the Cavs are still less likely (13%), and there is more uncertainty.



# WWWWCB: Will Warriors Win With Curry Back?

## 10 May 2016

Curry has returned from his injury, and after a slow shooting start, had the highest-scoring overtime period in the history of the NBA. Meanwhile, the Thunder lead the Spurs, 3-2, and the Cavaliers have been dominant in the East, hitting a historic number of 3-point shots. Here is my revised outlook: 

In [17]:
playoffs('Warriors',
     [('Rockets',  0.70, 4, 1),
      ('Blazers',  0.67, 3, 1),
      ('Spurs',    0.60),
      ('Cavs',     0.55)])

Warriors vs Rockets  100%; through here: 100%
Warriors vs Blazers   96%; through here:  96%
Warriors vs Spurs     71%; through here:  68%
Warriors vs Cavs      61%; through here:  42%


In [18]:
playoffs('Spurs',
     [('Memphis',  0.83, 4, 0),
      ('Thunder',  0.60, 2, 3),
      ('Warriors', 0.40),
      ('Cavs',     0.50)])

Spurs vs Memphis  100%; through here: 100%
Spurs vs Thunder   36%; through here:  36%
Spurs vs Warriors  29%; through here:  10%
Spurs vs Cavs      50%; through here:   5%


In [19]:
playoffs('Thunder',
     [('Dallas',   0.83, 4, 1),
      ('Spurs',    0.40, 3, 2),
      ('Warriors', 0.40),
      ('Cavs',     0.45)])

Thunder vs Dallas   100%; through here: 100%
Thunder vs Spurs     64%; through here:  64%
Thunder vs Warriors  29%; through here:  19%
Thunder vs Cavs      39%; through here:   7%


In [20]:
playoffs('Cavs',
     [('Pistons',  0.83, 4, 0),
      ('Hawks',    0.60, 4, 0),
      ('Raptors',  0.65),
      ('GS/SA/OK', 0.45)])

Cavs vs Pistons  100%; through here: 100%
Cavs vs Hawks    100%; through here: 100%
Cavs vs Raptors   80%; through here:  80%
Cavs vs GS/SA/OK  39%; through here:  31%


So overall, from the start of the playoffs up to May 10th, I have:

- **Warriors:** Dropped from 50% to 26% with Curry's injury, and rebounded to 42%. 
- **Spurs:** Dropped from 20% to 5% after falling behind Thunder.
- **Thunder:** Increased to 7%.
- **Cavs:** Increased to 31%.

# Time to Panic Yet?

## 17 May 2016

The Thunder finished off the Spurs and beat the Warriors in game 1. Are the Thunder, like the Cavs, peaking at just the right time, after an inconsistant regular season? Is it time for Warriors fans to panic?

Sure, the Warriors were down a game twice in last year's playoffs and came back to win both times. Sure, the Warriors are still 3-1 against the Thunder this year, and only lost two games all season to elite teams (Spurs, Thunder, Cavs, Clippers, Raptors). But the Thunder are playing at a top level. Here's my update, showing that the loss cost the Warriors 5%:

In [21]:
playoffs('Warriors',
     [('Rockets',  0.70, 4, 1),
      ('Blazers',  0.67, 4, 1),
      ('Thunder',  0.63, 0, 1),
      ('Cavs',     0.55)])

Warriors vs Rockets  100%; through here: 100%
Warriors vs Blazers  100%; through here: 100%
Warriors vs Thunder   61%; through here:  61%
Warriors vs Cavs      61%; through here:  37%


# Yet!

## 24 May 2016

The Thunder won two in a row (first time the Warriors had lost two in a row all year), putting the Warriors down 3-1. And the Cavs are looking mortal, losing two to the Raptors. So now it looks to me like the Thunder are favorites to win it all:

In [22]:
playoffs('Warriors',
     [('Rockets',  0.70, 4, 1),
      ('Blazers',  0.67, 4, 1),
      ('Thunder',  0.55, 1, 3),
      ('Cavs',     0.55)])

Warriors vs Rockets  100%; through here: 100%
Warriors vs Blazers  100%; through here: 100%
Warriors vs Thunder   17%; through here:  17%
Warriors vs Cavs      61%; through here:  10%


In [23]:
playoffs('Cavs',
     [('Pistons',  0.83, 4, 0),
      ('Hawks',    0.60, 4, 0),
      ('Raptors',  0.55, 2, 2),
      ('Thunder',  0.45)])

Cavs vs Pistons  100%; through here: 100%
Cavs vs Hawks    100%; through here: 100%
Cavs vs Raptors   57%; through here:  57%
Cavs vs Thunder   39%; through here:  23%


In [24]:
playoffs('Thunder',
     [('Dallas',   0.83, 4, 1),
      ('Spurs',    0.40, 4, 2),
      ('Warriors', 0.45, 3, 1),
      ('Cavs',     0.55)])

Thunder vs Dallas   100%; through here: 100%
Thunder vs Spurs    100%; through here: 100%
Thunder vs Warriors  83%; through here:  83%
Thunder vs Cavs      61%; through here:  51%


# The 2016 Finals

## 1 June 2016

The Warriors completed their comeback against the Thunder, putting them in a great position to win this year (and they are already established as [favorites for next year](http://www.foxsports.com/nba/story/golden-state-warriors-title-favorites-cleveland-cavaliers-odds-2016-17-053016)). Rather than update the odds after each game of the finals, I'll just repeat the table (with the note that I think the Warriors are somewhere in the 60% range for each game and thus about 70% to win the finals):


In [25]:
series_table()

W-L | Game Win Percentage
    |  20%  25%  30%  35%  40%  45%  50%  55%  60%  65%  70%  75%  80%
----+-----------------------------------------------------------------
0-0 |   3%   7%  13%  20%  29%  39%  50%  61%  71%  80%  87%  93%  97%
0-1 |   2%   4%   7%  12%  18%  26%  34%  44%  54%  65%  74%  83%  90%
0-2 |   1%   2%   3%   5%   9%  13%  19%  26%  34%  43%  53%  63%  74%
0-3 |   0%   0%   1%   2%   3%   4%   6%   9%  13%  18%  24%  32%  41%
----+-----------------------------------------------------------------
1-0 |  10%  17%  26%  35%  46%  56%  66%  74%  82%  88%  93%  96%  98%
1-1 |   6%  10%  16%  24%  32%  41%  50%  59%  68%  76%  84%  90%  94%
1-2 |   3%   5%   8%  13%  18%  24%  31%  39%  48%  56%  65%  74%  82%
1-3 |   1%   2%   3%   4%   6%   9%  12%  17%  22%  27%  34%  42%  51%
----+-----------------------------------------------------------------
2-0 |  26%  37%  47%  57%  66%  74%  81%  87%  91%  95%  97%  98%  99%
2-1 |  18%  26%  35%  44%  52%  61%  69%  76%  82% 